# SO(3), the group of rotations in 3D space
## Introduction
SO(3), the 3D rotation group, is the group of all roations about the origin of three-dimensional Euclidian space $\mathbb{R}^3$ under the operation of composition ([wiki](https://en.wikipedia.org/wiki/3D_rotation_group)). 

- A rotation about the origin preserves the origin, euclidian distance and orientation.
- Composing two rotations results in another rotation.
- Every rotation has a unique inverse rotation.
- The identity map satisfies the definition of a rotation.

Therefore the set of all rotations is a group under composition.

## Representing rotations

In [2]:
# TODO matrices, vectors, quaternions
class SO3():
    def __init__(self, R):
        self.R = R

## SO(3) in Lie theory
The rotation group has a natural structure as a manifold for which the group operations are smoothly differentiable, so it is a Lie group.

### Lie algebra
Associated with every Lie group is its Lie algebra, a linear space of the same dimension as the Lie group, closed under a bilinear alternating product called the Lie bracket. 

The Lie algebra of SO(3) is denoted by $\mathfrak {so}(3)$ and consists of all skew-symmetric 3 × 3 matrices. 

The Lie algebra $\mathfrak {so}(3)$ is isomorphic to the Lie algebra  $\mathbb{R}^3$. Under this isomorphism, an Euleur vector $\omega \in \mathbb{R}^3$ corresponds to the linear map $\tilde{\omega} (u) = \omega \times u$.

In practice, a suitable basis to use $\mathfrak {so}(3)$ as a 3-dimensional vector space is
$$
\boldsymbol{L}_x = \begin{bmatrix}0&0&0\\0&0&-1\\0&1&0\end{bmatrix}, \quad
\boldsymbol{L}_y = \begin{bmatrix}0&0&1\\0&0&0\\-1&0&0\end{bmatrix}, \quad
\boldsymbol{L}_z = \begin{bmatrix}0&-1&0\\1&0&0\\0&0&0\end{bmatrix}.
$$
We can then identify any matrix of in this Lie algebra with an Euler vector $\omega = (x, y, z) \in \mathbb{R}^3$.

$$
\widehat{\boldsymbol{\omega}} =\boldsymbol{\omega}\cdot \boldsymbol{L} = x \boldsymbol{L}_x + y \boldsymbol{L}_y + z \boldsymbol{L}_z =\begin{bmatrix}0&-z&y\\z&0&-x\\-y&x&0\end{bmatrix} \in \mathfrak{so}(3)
$$

We can go from the skew symetric notations to euler angles and back using the skew and vee operators.

In [30]:
def skew(v):
    return [
        [0, -v[2], v[1]], 
        [v[2], 0, -v[0]],
        [-v[1], v[0], 0]]

def vee(W):
    return [-W[1][2], W[0][2], -W[0][1]]

#### Logarithm map $\log : \operatorname{SO}(3) \to \mathfrak{so}(3)$
Given $R \in \operatorname{SO}(3)$, let $A = \frac{1}{2} (R - R^T)$ and let $\|A\| = \sqrt{-\frac{1}{2}\operatorname{Tr}\left(A^2\right)}$. Then the logarithm of $R$ is given by 
$$
\log R = \frac{\sin^{-1}\|A\|}{\|A\|}A
$$

In [8]:
import numpy as np
def log_map(R):
    """
    Returns the skew matrix in lie algebra associated to the 
    SO3 rotation matrix R
    """
    A = (R - R.T) / 2
    norm = np.sqrt(-np.trace(A @ A)/2)
    return ((np.arcsin(norm)) / norm) * A

In [17]:
from scipy.spatial.transform import Rotation as Rot
RR = Rot.from_rotvec(np.pi/2 * np.array([0, 0, 1]))
R = RR.as_matrix()
so = log_map(R)
v = vee(so)
v, RR.as_euler("xyz")

([-0.0, 0.0, 1.5707963267948966], array([0.        , 0.        , 1.57079633]))

#### Exponential map $\exp : \mathfrak{so}(3) \to \operatorname{SO}(3)$
Since $\operatorname{SO}(3)$ is a matrix group, the exponential map is the matrix exponential series
$$
\begin{cases}
\exp : \mathfrak{so}(3) \to \operatorname{SO}(3) \\
A \mapsto e^A = \sum_{k=0}^\infty \frac{1}{k!} A^k
 = I + A + \tfrac{1}{2} A^2 + \cdots.
\end{cases}
$$
$A \in \mathfrak{so}(3)$ being by definition a skew-symetric matrix.

In [46]:
def exp_map_generic(A):
    exp = np.eye(3)
    for k in range(1, 50):
        exp += (1/np.math.factorial(k)) * np.linalg.matrix_power(A, k)
    return exp

test = skew(v)

exp_map_generic(test)


array([[ 4.26480464e-17, -1.00000000e+00,  0.00000000e+00],
       [ 1.00000000e+00,  4.26480464e-17,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

However as every $A \in \mathfrak{so}(3)$ is associated to a vector $\omega = \theta u$, a direct exponentiation is possible, and yields

$$
\begin{align}
 \exp(\tilde{\boldsymbol{\omega}})
   &= \exp(\theta(\boldsymbol{u\cdot L}))
    = \exp\left(\theta \begin{bmatrix} 0 & -z & y \\ z & 0 & -x \\ -y & x & 0 \end{bmatrix}\right)\\[4pt]
   &= \boldsymbol{I} + 2cs(\boldsymbol{u\cdot L}) + 2s^2 (\boldsymbol{u \cdot L})^2 \\[4pt]
   &= \begin{bmatrix}
        2 \left(x^2 - 1\right) s^2 + 1 & 2 x y s^2 - 2 z c s & 2 x z s^2 + 2 y c s \\
        2 x y s^2 + 2 z c s & 2 \left(y^2 - 1\right) s^2 + 1 & 2 y z s^2 - 2 x c s \\
        2 x z s^2 - 2 y c s & 2 y z s^2 + 2 x c s & 2 \left(z^2 - 1\right) s^2 + 1
      \end{bmatrix},
\end{align}
$$

In [47]:
def exp_map_euler(w):
   
    w = np.array(vee(w))
    theta = np.linalg.norm(w)
    w = w / theta
    x = w[0]
    y = w[1]
    z = w[2]

    c = np.cos(theta / 2)
    s = np.sin(theta / 2)

    return [
        [2*(x**2 - 1)*s**2 + 1, 2*x*y*(s**2) - 2*z*c*s, 2*x*z*s**2 + 2*y*c*s],
        [2*x*y*s**2 + 2*z*c*s, 2*(y**2-1)*s**2+1, 2*y*z*s**2 - 2*x*c*s],
        [2*x*z*s**2 - 2*y*c*s, 2*y*z*s**2 + 2*x*c*s, 2*(z**2-1)*s**2 + 1]
    ]

np.array(exp_map_euler(test))

array([[ 2.22044605e-16, -1.00000000e+00,  0.00000000e+00],
       [ 1.00000000e+00,  2.22044605e-16,  0.00000000e+00],
       [-0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])